In [1]:
!pip install torch torchvision torchaudio
!pip install ogb
!pip install torch-geometric

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [2]:
import os, random, numpy as np, torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import to_networkx
import networkx as nx
import wandb
from kaggle_secrets import UserSecretsClient

# W&B login via Kaggle secrets
user_secrets = UserSecretsClient()
wandb.login(key=user_secrets.get_secret("wandb-key"))

EPOCHS = int(os.getenv("EPOCHS", "50"))
NUM_SEEDS = int(os.getenv("NUM_SEEDS", "30"))
PROJECT = "GCN Graph classification(ENZYMES)_Gal"
ENTITY = "noynetanel1-tel-aviv-university"

def set_seed(s):
    random.seed(s)
    np.random.seed(s)
    torch.manual_seed(s)
    torch.cuda.manual_seed_all(s)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

class CentralityEncoder(nn.Module):
    def __init__(self, in_dim=1, out_dim=16):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, 32),
            nn.ReLU(),
            nn.Linear(32, out_dim)
        )
    def forward(self, c):
        return self.net(c)

class GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, cent_dim=16):
        super().__init__()
        self.cent_enc = CentralityEncoder(1, cent_dim)
        self.conv1 = GCNConv(in_channels + cent_dim, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = nn.Linear(hidden_channels, out_channels)
    def forward(self, x, cent, edge_index, batch):
        x = torch.cat([x, self.cent_enc(cent)], dim=1)
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        return F.log_softmax(self.lin(x), dim=1)

def _minmax_norm(t):
    if len(t) == 0:
        return t
    vmin, vmax = t.min(), t.max()
    if (vmax - vmin) > 0:
        return (t - vmin) / (vmax - vmin)
    return torch.zeros_like(t)

def make_centrality_transform():
    def transform(data):
        if getattr(data, "x", None) is None:
            data.x = torch.ones((data.num_nodes, 1), dtype=torch.float)
        G = to_networkx(data, to_undirected=True)
        cent = torch.zeros((data.num_nodes, 1), dtype=torch.float)
        if G.number_of_nodes() > 0:
            try:
                ev = nx.eigenvector_centrality_numpy(G)
            except Exception:
                ev = {n: 0.0 for n in G.nodes}
            ev_tensor = torch.tensor([ev.get(n, 0.0) for n in range(data.num_nodes)], dtype=torch.float)
            cent[:, 0] = _minmax_norm(ev_tensor)
        data.cent = cent
        return data
    return transform

def run_seed(seed):
    set_seed(seed)
    dataset = TUDataset(root="data/TUDataset", name="ENZYMES", transform=make_centrality_transform())
    dataset = dataset.shuffle()
    train_dataset = dataset[:len(dataset)//2]
    test_dataset = dataset[len(dataset)//2:]
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = GCN(dataset.num_features if dataset.num_features else 1, 64, dataset.num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    wandb.init(
        project=PROJECT,
        entity=ENTITY,
        name=f"GCN_Centrality_PROTEINS_seed_{seed}",
        group="GCN_PROTEINS_Centrality",
        job_type="centrality_only",
        reinit=True,
        config={
            "model": "GCN",
            "hidden_channels": 64,
            "epochs": EPOCHS,
            "lr": 0.001,
            "dataset": "PROTEINS",
            "task": "Centrality_eigenvector",
            "seed": seed
        }
    )

    def train():
        model.train()
        total_loss, correct, count = 0.0, 0, 0
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()
            out = model(data.x, data.cent, data.edge_index, data.batch)
            loss = F.nll_loss(out, data.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
            count += data.num_graphs
        return total_loss / len(train_loader), correct / count

    def test(loader):
        model.eval()
        correct, count = 0, 0
        for data in loader:
            data = data.to(device)
            out = model(data.x, data.cent, data.edge_index, data.batch)
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
            count += data.num_graphs
        return correct / count

    for epoch in range(1, EPOCHS + 1):
        loss, train_acc = train()
        test_acc = test(test_loader)
        print(f"Seed {seed} | Epoch {epoch:03d} | Loss: {loss:.4f} | Train Acc: {train_acc:.4f} | Test Acc: {test_acc:.4f}")
        wandb.log({
            "epoch": epoch,
            "loss": loss,
            "train_acc": train_acc,
            "test_acc": test_acc
        })

    wandb.finish()

if __name__ == "__main__":
    for s in range(1, NUM_SEEDS + 1):
        run_seed(s)


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: galsalman (noynetanel1-tel-aviv-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
Processing...
Done!
/usr/local/lib/python3.11/dist-packages/networkx/algorithms/centrality/eigenvector.py:352: RuntimeWarning: k >= N - 1 for N * N square matrix. Attempting to use scipy.linalg.eig instead.
  _, eigenvector = sp.sparse.linalg.eigs(
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250914_174457-c4d2k8it
wandb: Run `wandb offlin

Seed 1 | Epoch 001 | Loss: 1.8039 | Train Acc: 0.1667 | Test Acc: 0.1667
Seed 1 | Epoch 002 | Loss: 1.7929 | Train Acc: 0.2000 | Test Acc: 0.1633
Seed 1 | Epoch 003 | Loss: 1.7875 | Train Acc: 0.2033 | Test Acc: 0.1733
Seed 1 | Epoch 004 | Loss: 1.7851 | Train Acc: 0.1767 | Test Acc: 0.1433
Seed 1 | Epoch 005 | Loss: 1.7812 | Train Acc: 0.1967 | Test Acc: 0.1500
Seed 1 | Epoch 006 | Loss: 1.7772 | Train Acc: 0.1700 | Test Acc: 0.1700
Seed 1 | Epoch 007 | Loss: 1.7738 | Train Acc: 0.1500 | Test Acc: 0.1333
Seed 1 | Epoch 008 | Loss: 1.7701 | Train Acc: 0.1967 | Test Acc: 0.1700
Seed 1 | Epoch 009 | Loss: 1.7679 | Train Acc: 0.2300 | Test Acc: 0.1800
Seed 1 | Epoch 010 | Loss: 1.7624 | Train Acc: 0.2000 | Test Acc: 0.1333
Seed 1 | Epoch 011 | Loss: 1.7576 | Train Acc: 0.2133 | Test Acc: 0.1267
Seed 1 | Epoch 012 | Loss: 1.7483 | Train Acc: 0.1767 | Test Acc: 0.1567
Seed 1 | Epoch 013 | Loss: 1.7560 | Train Acc: 0.2300 | Test Acc: 0.1933
Seed 1 | Epoch 014 | Loss: 1.7509 | Train Acc: 0.19

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss █▇▇▇▆▆▅▅▄▄▄▃▄▄▃▃▃▃▃▃▃▃▂▂▂▂▃▂▂▂▂▂▂▂▂▁▁▁▁▁
wandb:  test_acc ▄▄▅▃▃▂▅▅▂▂▆▃▃▂▆▂▁▁▁▄▄▄▄▅▆▃▃▅▆▅▅▅▅▆▆▆▇▆█▆
wandb: train_acc ▂▅▅▃▄▁▄▇▅▅▇▄▃▃▇▃▃▃▃▅▄▆▇▇▄█▅▅▇▅▇▇▅▆▅▆▇▇██
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 1.71399
wandb:  test_acc 0.2
wandb: train_acc 0.24333
wandb: 
wandb: 🚀 View run GCN_Centrality_PROTEINS_seed_1 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal/runs/c4d2k8it
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250914_174457-c4d2k8it/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/work

Seed 2 | Epoch 001 | Loss: 1.7969 | Train Acc: 0.1700 | Test Acc: 0.1633
Seed 2 | Epoch 002 | Loss: 1.7922 | Train Acc: 0.1600 | Test Acc: 0.1800
Seed 2 | Epoch 003 | Loss: 1.7900 | Train Acc: 0.2133 | Test Acc: 0.1800
Seed 2 | Epoch 004 | Loss: 1.7857 | Train Acc: 0.2200 | Test Acc: 0.2000
Seed 2 | Epoch 005 | Loss: 1.7817 | Train Acc: 0.2067 | Test Acc: 0.2300
Seed 2 | Epoch 006 | Loss: 1.7822 | Train Acc: 0.2400 | Test Acc: 0.2067
Seed 2 | Epoch 007 | Loss: 1.7767 | Train Acc: 0.2100 | Test Acc: 0.1867
Seed 2 | Epoch 008 | Loss: 1.7734 | Train Acc: 0.2500 | Test Acc: 0.2233
Seed 2 | Epoch 009 | Loss: 1.7716 | Train Acc: 0.1800 | Test Acc: 0.2000
Seed 2 | Epoch 010 | Loss: 1.7683 | Train Acc: 0.1667 | Test Acc: 0.2233
Seed 2 | Epoch 011 | Loss: 1.7687 | Train Acc: 0.1800 | Test Acc: 0.2433
Seed 2 | Epoch 012 | Loss: 1.7642 | Train Acc: 0.2567 | Test Acc: 0.2067
Seed 2 | Epoch 013 | Loss: 1.7590 | Train Acc: 0.2700 | Test Acc: 0.2033
Seed 2 | Epoch 014 | Loss: 1.7571 | Train Acc: 0.22

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
wandb:      loss ██▇▇▇▆▆▆▆▆▅▅▄▄▅▄▄▄▄▄▃▃▃▃▃▂▂▃▂▂▁▁▂▂▁▁▁▂▂▁
wandb:  test_acc ▁▂▂▄▇▃▆▄▆█▄█▇▄▇▄▄▆▅▇▅▇▆▄█▇▇▇▇█▆▇▆▆█▅▇▆▆▇
wandb: train_acc ▂▁▄▄▅▆▂▁▂▆▅▅▅▄▅▆▅▅▆▆▇█▆▅▇▆▆▇▇▇▆▅▅▇█▆▆▇▅▇
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 1.70832
wandb:  test_acc 0.23
wandb: train_acc 0.26667
wandb: 
wandb: 🚀 View run GCN_Centrality_PROTEINS_seed_2 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal/runs/bvdugyhy
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250914_174600-bvdugyhy/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/wor

...

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
wandb:      loss █▇▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▄▄▄▄▃▂▃▃▂▂▂▃▂▂▂▂▁▂▁▁
wandb:  test_acc █▁▂▃▅▄▄▅▂▄▆▄▅▅▆█▄▃▃▄▅▅▅▃▆▄▅▅▆▅▆▃▁▄▇▃▇▄▄▂
wandb: train_acc ▁▃▃▃▅▄▄▅▅▄▄▆▄▆▇▇▇▆▆▇▇▇▆▇▇▆▆▇█▇▇▇▇▆██▆▇█▇
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 1.69397
wandb:  test_acc 0.16
wandb: train_acc 0.27
wandb: 
wandb: 🚀 View run GCN_Centrality_PROTEINS_seed_28 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal/runs/3ln6j4pr
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250914_181143-3ln6j4pr/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/worki

Seed 29 | Epoch 001 | Loss: 1.8016 | Train Acc: 0.1567 | Test Acc: 0.1567
Seed 29 | Epoch 002 | Loss: 1.7916 | Train Acc: 0.1733 | Test Acc: 0.1767
Seed 29 | Epoch 003 | Loss: 1.7926 | Train Acc: 0.1500 | Test Acc: 0.1567
Seed 29 | Epoch 004 | Loss: 1.7868 | Train Acc: 0.1733 | Test Acc: 0.1767
Seed 29 | Epoch 005 | Loss: 1.7893 | Train Acc: 0.1533 | Test Acc: 0.1700
Seed 29 | Epoch 006 | Loss: 1.7861 | Train Acc: 0.1733 | Test Acc: 0.2300
Seed 29 | Epoch 007 | Loss: 1.7809 | Train Acc: 0.2167 | Test Acc: 0.2133
Seed 29 | Epoch 008 | Loss: 1.7839 | Train Acc: 0.1633 | Test Acc: 0.1567
Seed 29 | Epoch 009 | Loss: 1.7794 | Train Acc: 0.1567 | Test Acc: 0.1433
Seed 29 | Epoch 010 | Loss: 1.7763 | Train Acc: 0.1600 | Test Acc: 0.1767
Seed 29 | Epoch 011 | Loss: 1.7750 | Train Acc: 0.2033 | Test Acc: 0.1600
Seed 29 | Epoch 012 | Loss: 1.7805 | Train Acc: 0.1967 | Test Acc: 0.1467
Seed 29 | Epoch 013 | Loss: 1.7717 | Train Acc: 0.2000 | Test Acc: 0.2300
Seed 29 | Epoch 014 | Loss: 1.7661 | T

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss █▇▇▇▇▆▇▆▆▆▆▅▅▆▅▄▅▄▄▄▄▄▃▃▃▃▂▃▃▃▂▂▂▁▂▂▁▁▁▁
wandb:  test_acc ▂▃▂▃▂▅▂▃▂▁▆▆▄▅▅▅▅▄▃▇▄▅█▄▇▄▇▆▄▇▇▆▇▆▇█▇▇▇▇
wandb: train_acc ▁▃▁▃▁▆▂▁▂▅▅▆▅▅▃▃▆▅▃▅▃▆▇▆▅▅▄▅▇▆▇▇██▇▆▇▇█▇
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 1.71737
wandb:  test_acc 0.24667
wandb: train_acc 0.23667
wandb: 
wandb: 🚀 View run GCN_Centrality_PROTEINS_seed_29 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal/runs/khl7vn9g
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250914_181241-khl7vn9g/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle

Seed 30 | Epoch 001 | Loss: 1.7914 | Train Acc: 0.1667 | Test Acc: 0.1533
Seed 30 | Epoch 002 | Loss: 1.7849 | Train Acc: 0.1800 | Test Acc: 0.1533
Seed 30 | Epoch 003 | Loss: 1.7851 | Train Acc: 0.2167 | Test Acc: 0.1933
Seed 30 | Epoch 004 | Loss: 1.7851 | Train Acc: 0.1867 | Test Acc: 0.1433
Seed 30 | Epoch 005 | Loss: 1.7822 | Train Acc: 0.1933 | Test Acc: 0.1867
Seed 30 | Epoch 006 | Loss: 1.7798 | Train Acc: 0.2200 | Test Acc: 0.1600
Seed 30 | Epoch 007 | Loss: 1.7779 | Train Acc: 0.1700 | Test Acc: 0.1600
Seed 30 | Epoch 008 | Loss: 1.7771 | Train Acc: 0.1900 | Test Acc: 0.1733
Seed 30 | Epoch 009 | Loss: 1.7801 | Train Acc: 0.2200 | Test Acc: 0.1800
Seed 30 | Epoch 010 | Loss: 1.7765 | Train Acc: 0.2200 | Test Acc: 0.1400
Seed 30 | Epoch 011 | Loss: 1.7731 | Train Acc: 0.1833 | Test Acc: 0.1400
Seed 30 | Epoch 012 | Loss: 1.7771 | Train Acc: 0.1833 | Test Acc: 0.1500
Seed 30 | Epoch 013 | Loss: 1.7656 | Train Acc: 0.2300 | Test Acc: 0.1933
Seed 30 | Epoch 014 | Loss: 1.7687 | T

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss █▇▇▇▇▇▆▇▆▆▆▅▅▅▅▅▄▅▄▃▄▄▄▄▄▄▃▃▄▃▄▃▃▂▃▃▃▂▂▁
wandb:  test_acc ▂▂▅▁▅▃▄▄▁▁▅▃▂▃▅▅▂▃▃▃▄▄▂▄▂▅▄▄▄▅▅▆▆█▆█▅▇█▆
wandb: train_acc ▁▂▆▃▄▁▃▆▆▃▆▅▃▄▆▄▄▄▄▆▅▄▅▄▅▄▄▅▅▅▇▅▇▆▃▇█▇█▅
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 1.72628
wandb:  test_acc 0.19667
wandb: train_acc 0.20667
wandb: 
wandb: 🚀 View run GCN_Centrality_PROTEINS_seed_30 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal/runs/e9kl3oiu
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250914_181343-e9kl3oiu/logs
